In [1]:
apicsv_path = "apiFunList.csv"

In [2]:
api2index = {}
index2api = {}
with open(apicsv_path, 'r') as f:
    api_list = f.read().split("\n")
for api in api_list[1:-1]:
    index, apifun = api.split(",")
    api2index[apifun] = index
    index2api[index] = apifun
api_list = index2api.values()

In [15]:
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
import tqdm

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 232k/232k [00:00<00:00, 321kB/s]  
d:\miniconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sivan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 14.1kB/s]
Downloading: 100%|██

In [9]:
def get_encoded_api(api):
    encoded_dict = tokenizer.encode_plus(
                        api,    # Sentence to encode.
                        add_special_tokens = False, # Not Add '[CLS]' and '[SEP]'
                        return_attention_mask = False,   
                        return_tensors = 'pt'    
                )
    return encoded_dict['input_ids']

api_tokenized_ids = [get_encoded_api(api) for api in api_list]


In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [27]:

import os
from tqdm import tqdm
api_tmber_path = 'api_embeddings2.np'
#如果文件存在，就删除
if os.path.exists(api_tmber_path):
    os.remove(api_tmber_path)
length = len(api_tokenized_ids)
api_embed_array = np.empty((length,768))
for i in tqdm(range(0,len(api_tokenized_ids))):
    input = api_tokenized_ids[i]
    input = input.to(device)
    outputs = model(input)
    outputs = outputs[0].mean(dim=1, keepdim=False)
    outputs = outputs.squeeze(0)
    api_embed_array[i] = outputs.cpu().detach().numpy()

100%|██████████| 15938/15938 [01:31<00:00, 173.68it/s]


In [28]:
with open('api_embeddings2.np','wb') as f:
    np.save(f,api_embed_array)

In [15]:
# 加载api_embed文件，加载api_csv文件，生成api2token字典
import numpy as np

api_embed_path = "../data/api_embed_ae_10.npy"
api_csv_path = "../data/apiFunList.csv"
api2token_path = "../data/api2token.npy"

api_embedding = np.load(api_embed_path, allow_pickle=True)
api2token = {}

with open(api_csv_path, 'r') as f:
    api_list = f.read().split("\n")
for api in api_list[1:-1]:
    # api_list[1:-1] 删去表头和结尾的空格
    # index从0开始
    index, apifun = api.split(",")
    api2token[apifun] = api_embedding[int(index)]
# 将空api的embedding置为0,长度为api_embedding文件的维度
api2token[""] = np.zeros(api_embedding.shape[1])
np.save(api2token_path, api2token)